- Question 3. Consider the dataset movies.json, which is attached in the submission link. Each record stores the information of a movies: name, year of release, list of actors and list of genres. Write a PySpark application that uses RDD/DataFrame to fulfill the following requirements.
    - a. Read the dataset to a DataFrame. Print out the schema and the number of records.
    - b. Print out the number of movies released during 2012 and 2015 (included)
    - c. Print out the number of actors (distinct) available in the dataset
    - d. Show the year that has the highest number of movies released
    - e. List all movies whose names contain the word “superman” (case-insensitive). Identify the number of genres that these movies are categorized to
    - f. Sort the names of actors in descending order of the total number of genres of all movies that they participated

In [1]:
import findspark
findspark.init("D:\spark\spark-3.1.1-bin-hadoop3.2")
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import size
from pyspark.sql.functions import max, col

spark = SparkSession.builder.getOrCreate()

#### Load the moives.json into dataframe df

In [2]:
df = spark.read.json("movies.json")
df.printSchema()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



cast,genres,title,year
[],[],After Dark in Cen...,1900
[],[],Boarding School G...,1900
[],[],Buffalo Bill's Wi...,1900
[],[],Caught,1900
[],[],Clowns Spinning Hats,1900
[],"[Short, Documentary]",Capture of Boer B...,1900
[],[],The Enchanted Dra...,1900
[Paul Boyton],[],Feeding Sea Lions,1900
[],[Comedy],How to Make a Fat...,1900
[],[],New Life Rescue,1900


#### c. Print out the number of actors (distinct) available in the dataset

- Using select function to select to column "cast" with the condition "the number of actors of this movie is higher than 0" by using the where function check the size of each cell in "cast" column higher than 0. Then save the result into actors.

In [3]:
actors = df.select("cast").where(size(df.cast) > 0)
actors

cast
[Paul Boyton]
[Ching Ling Foo]
[May Clark]
[William Carrington]
[J. Stuart Blackt...
[J. Stuart Blackton]
[William S. Hart]
"[William Craven, ..."
[Unknown]
"[Bertha Regustus,..."


- Collecting the data of each cell in "cast" column then save it into all_ActorsOfData
- Creating an array all_ActorsOfData to save each actor that participated to each movie.
- Using dict.formkeys to get all distinct names from array all_ActorsOfData then save it into distinct_actors.

In [4]:
all_ActorsOfData = actors.select("cast").collect()
all_ActorsOfData = [actor for actorsOfMovie in all_ActorsOfData for actor in actorsOfMovie.cast]
distinct_actors = list(dict.fromkeys(all_ActorsOfData))

print("Number of actors (distinct) available in the dataset: ", len(distinct_actors))

Number of actors (distinct) available in the dataset:  15531


- Additionally, we can use the explode function to extract the array of actors of each movie into a new column "samples".
- Then, we count the distinct actors from the column "samples"

In [5]:
from pyspark.sql.functions import explode

all_ActorsOfData1 = actors.select("cast", explode("cast").alias("samples"))

distinct_actors1 = all_ActorsOfData1.select("samples").distinct().count()
all_ActorsOfData1

cast,samples
[Paul Boyton],Paul Boyton
[Ching Ling Foo],Ching Ling Foo
[May Clark],May Clark
[William Carrington],William Carrington
[J. Stuart Blackt...,J. Stuart Blackton
[J. Stuart Blackt...,Florence Lawrence
[J. Stuart Blackton],J. Stuart Blackton
[William S. Hart],William S. Hart
"[William Craven, ...",William Craven
"[William Craven, ...",Florence Lawrence


In [6]:
print("Number of actors (distinct) available in the dataset: ", distinct_actors1)

Number of actors (distinct) available in the dataset:  15531


#### d. Show the year that has the highest number of movies released

- Using groupBy function by year and count the number of movies released of that year.

In [7]:
year_data = df.groupBy("year").count()

In [8]:
year_data

year,count
1950,443
1919,634
1936,504
1951,429
1958,281
1921,143
1983,140
1905,35
1972,140
1979,139


- Find the maximum number of movies realeased of the year_data then get that number saved it into tmp variable.

In [9]:
result = year_data.select(max("count"))
tmp = result.collect()[0]['max(count)']
tmp

634

- Using filter function to filter all the year had the exactly maximum movies released.

In [10]:
year_data.filter(col("count") == tmp)

year,count
1919,634


#### e. List all movies whose names contain the word “superman” (case-insensitive). Identify the number of genres that these movies are categorized to

- Using filter function to filter all the title of movie has the word "supper man" by using rlike function passed by the regex formular (?i) makes that title match case insensitive

In [11]:
movie_data = df.filter(df.title.rlike("(?i)superman"))

In [12]:
movie_data

cast,genres,title,year
"[Kirk Alyn, Noel ...",[],Superman,1948
"[Kirk Alyn, Lyle ...",[],Atom Man vs. Supe...,1950
"[George Reeves, P...",[Action],Superman and the ...,1951
[George Reeves],[Short],Stamp Day for Sup...,1954
"[Marlon Brando, G...",[Superhero],Superman,1978
"[Gene Hackman, Ch...","[Action, Adventure]",Superman II,1980
[Christopher Reev...,[Action],Superman III,1983
[Christopher Reev...,[Superhero],Superman IV: The ...,1987
"[Brandon Routh, K...",[Superhero],Superman Returns,2006
[The analyzation ...,[Documentary],Waiting for Superman,2010


In [13]:
movie_data.select("*", size(movie_data.genres).alias("number_of_genres"))

cast,genres,title,year,number_of_genres
"[Kirk Alyn, Noel ...",[],Superman,1948,0
"[Kirk Alyn, Lyle ...",[],Atom Man vs. Supe...,1950,0
"[George Reeves, P...",[Action],Superman and the ...,1951,1
[George Reeves],[Short],Stamp Day for Sup...,1954,1
"[Marlon Brando, G...",[Superhero],Superman,1978,1
"[Gene Hackman, Ch...","[Action, Adventure]",Superman II,1980,2
[Christopher Reev...,[Action],Superman III,1983,1
[Christopher Reev...,[Superhero],Superman IV: The ...,1987,1
"[Brandon Routh, K...",[Superhero],Superman Returns,2006,1
[The analyzation ...,[Documentary],Waiting for Superman,2010,1
